In [ ]:
# develop-hamming-distance-permutation.ipynb
#
# Bryan Daniels, Enrico Borriello
# 2023/9/19
#

In [3]:
import attattach as at
import numpy as np

In [ ]:
at.join_transitions()

In [4]:
def label_to_state (label, digits):
    return np.array(list(map(int,list(format(label,'0'+str(digits)+'b')))))

def state_to_label (state):
    return int(''.join(map(str,state)),2)

In [7]:
def hamming_distance(label1,label2,n):
    return np.sum(abs(label_to_state(label1,n) - label_to_state(label2,n)))

In [11]:
hamming_distance(1,3,n)

1

In [13]:
hamming_distance(1,5,n)

1

In [19]:
hamming_distance(21,21,n)

0

In [16]:
label_to_state(21,n)

array([1, 0, 1, 0, 1])

In [24]:
state_to_label([1,0,1,0,0])

20

In [ ]:
def labels_permutation_close(transitions):
    """
    randomly reassign state labels,
    with bias toward states that are closer in
    Hamming distance
    """
    # Generate a random permutation of the labels
    s = len(transitions)
    p = list(range(s))
    random.shuffle(p)
    # Create a new list of edges with the updated labels
    new_transitions = [(p[i], p[j]) for (i, j) in transitions]
    sorted_transitions = sorted(new_transitions, key=lambda x: x[0])
    return sorted_transitions

In [ ]:
def generate_landscape(num_nodes,landscape_structure,close=True):
    """
    Sample landscape structure: [[3,.25],[1,.50],[1,.05],[2,.20]]
    This corresponds to 4 attractors, of lengths 3, 1, 1, and 2,
    with relative basins sizes equal to 25%, 50%, 5%, and 20%
    """
    
    s = 2**num_nodes # total number of states in the attractor landscape

    # Read the structure of the attractor landscape
    lengths = [B[0] for B in landscape_structure]
    rel_sizes = [B[1] for B in landscape_structure]
    sizes = [int(rel_size*s) for rel_size in rel_sizes] # the last one might be wrong
    sizes[-1] = s-(sum(sizes)-sizes[-1]) # this fixes it
    # attractor states in each basin:
    num_att_states = [ landscape_structure[i][0] for i in range(len(landscape_structure)) ]
    
    # CONDITION 1:
    # 'The sum of the relative basin sizes needs to be 1'
    c1 = np.allclose(sum(rel_sizes),1.) 

    # CONDITION 2:
    # All the basins have at least size 1 
    # (For small n and small relative size of a basin, the product might result in zero states)
    c2 = np.prod([sizes[i] > 0 for i in range(len(sizes))])

    # CONDITION3:
    # There are at least as many states as attractor states
    c3 = np.sum(num_att_states) <= s

    # CONDITION4:
    # There are at least as many states as attractor states **in each individual basin**
    c4 = np.prod([ num_att_states[i] <= sizes[i] for i in range(len(landscape_structure)) ])

    # If all conditions are satisfied, proceed:
    if c1*c2*c3*c4:
    
        # generate the individual basins
        t = []
        for i in range(len(landscape_structure)):
            t.append(transitions(lengths[i], sizes[i]))

        # join them with the sequential relabeling 
        all_t = []
        for i in range(len(t)):
            all_t = join_transitions(all_t,t[i])
    
        if close:
            return labels_permutation_close(all_t)
        else:
            return labels_permutation(all_t)

    else:
        if not c1:
            print('ERROR: The sum of the relative basin sizes is not 1.')
        if not c2:
            print('ERROR: At least one basin has size 0.')
            print('       (relative size is too small for your n).')
        if not c3:
            print('ERROR: There are more attractor states than total states.')
        if not c4:
            print('ERROR: There are more attractor states than total states in at least one basin.')
        return None


In [2]:
landscape_structure = [[1,.25],[1,.50],[1,.05],[1,.20]]
n = 5
at.generate_landscape(n,landscape_structure)

[(0, 8),
 (1, 28),
 (2, 28),
 (3, 20),
 (4, 28),
 (5, 28),
 (6, 9),
 (7, 25),
 (8, 6),
 (9, 18),
 (10, 28),
 (11, 28),
 (12, 28),
 (13, 31),
 (14, 14),
 (15, 25),
 (16, 28),
 (17, 25),
 (18, 18),
 (19, 25),
 (20, 9),
 (21, 23),
 (22, 28),
 (23, 28),
 (24, 8),
 (25, 30),
 (26, 13),
 (27, 25),
 (28, 28),
 (29, 28),
 (30, 30),
 (31, 28)]